# Example 3-7: Calculating Dynamical Time
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 196-198

This notebook demonstrates converting a UTC time to various time systems.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy

Install `pandas` for visual data display:

In [2]:
!pip install pandas

Import libraries:

In [3]:
import pandas as pd
from datetime import timedelta
from IPython.display import HTML, display
from valladopy.mathtime.julian_date import convtime

## Problem Definition
---

GIVEN: &ensp;May 14, 2004, 10:43 Mountain Standard Time (UTC based)<br>
FIND: &emsp;$UT1$, $TAI$, $GPS$, $TT$, $TDB$, $T_{UT1}$, $T_{TT}$, $T_{TDB}$

In [4]:
year = 2004
month = 5
day = 14
hour = 10
minute = 43
second = 0

## Solution
---

We first find the time zone offset from UTC. **Table 3-2** summarizes the UTC offsets for selected time zones:

| Time Zone          | Standard   | Daylight Savings | Central Meridian |
|--------------------|------------|------------------|------------------|
| Atlantic           | UTC − 4ʰ   | UTC − 3ʰ         | −60°             |
| Eastern            | UTC − 5ʰ   | UTC − 4ʰ         | −75°             |
| Mountain           | UTC − 7ʰ   | UTC − 6ʰ         | −105°            |
| Alaska             | UTC − 9ʰ   | UTC − 8ʰ         | −135°            |
| Hawaii             | UTC − 10ʰ  |                  | −150°            |
| Central European   | UTC + 1ʰ   | UTC + 2ʰ         | 10°              |
| Moscow             | UTC + 4ʰ   |                  | 40°              |
| China              | UTC + 8ʰ   |                  | 100°             |
| Japan              | UTC + 9ʰ   |                  | 140°             |
| Australian Eastern | UTC + 10ʰ  | UTC + 11ʰ        | 146°             |

Since the date falls during Daylight Savings Time, the UTC offset is $6$ hours:

In [5]:
timezone = 6  # Mountain time, hours offset from UTC during daylight savings

We can then obtain the cumulative number of leap seconds added to UTC, $\Delta AT$, from the *Astronomic Almanac*:

$$
\Delta AT = 32.0^{\text{S}}
$$

The historical list of leap seconds can also be found [here](https://maia.usno.navy.mil/ser7/tai-utc.dat).

In [6]:
dat = 32  # seconds

$\Delta UT1$ is observed and isn't listed directly; it can be obtained from the USNO's IERS [final data](https://maia.usno.navy.mil/ser7/finals.all):

In [7]:
dut1 = -0.463326  # seconds

**Algorithm 16** summarizes the process for converting to different time systems.

First we calculate UT1, the form of Universal Time that reflects the Earth's actual rotation:

$$
UT1 = UTC + \Delta UT1
$$

where UTC is the Coordinated Universal Time and $\Delta UT1$ is the difference between UT1 and UTC, published by the IERS to account for variations in Earth's rotation.

Next we find the atomic time:
$$
TAI = UTC + \Delta AT
$$

GPS time can then be calculated with:

$$
GPS = UTC + \Delta AT - 19^{\text{s}} = TAI - 19^{\text{s}}
$$

Dynamical time is then:

$$
TT = TAI + 32.184^{\text{s}}
$$

The Julian centuries of TT can be found using **Eq. 3-43**:

$$
T_{TT} = \frac{JD_{TT}-2,451,545.0}{36,525}
$$

where the Julian date of TT ($JD_{TT}$) can be found from the given TT time in $DMYHMS$.

Then, the barycentric time is found with:

$$
\begin{aligned}
\text{TDB} = \text{TT}
&+ 0.001\,657^{\text{S}} \sin(628.3076\, T_{TT} + 6.2401) \\
&+ 0.000\,022 \sin(575.3385\, T_{TT} + 4.2970) \\
&+ 0.000\,014 \sin(1256.6152\, T_{TT} + 6.1969) \\
&+ 0.000\,005 \sin(606.9777\, T_{TT} + 4.0212) \\
&+ 0.000\,005 \sin(52.9691\, T_{TT} + 0.4444) \\
&+ 0.000\,002 \sin(21.3299\, T_{TT} + 5.5431) \\
&+ 0.000\,010\, T_{TT} \sin(628.3076\, T_{TT} + 4.2490)
\end{aligned}
$$

Finally, the Julian centuries of TDB is:

$$
T_{TDB} = \frac{JD_{TDB}-2,451,545.0}{36,525}
$$

and for UT1:

$$
T_{UT1} = \frac{JD_{UT1}-2,451,545.0}{36,525}
$$


where the Julian dates TDB ($JD_{TDB}$) and UT1 ($JD_{UT1}$) are found from their respective times

Use the `convtime` routine to find all these quantities:

In [8]:
ut1, tut1, jdut1, jdut1frac, utc, tai, tt, ttt, jdtt, jdttfrac, tdb, ttdb, jdtdb, jdtdbfrac = convtime(
    year, month, day, hour, minute, second, timezone, dut1, dat
)
gps = tai - 19

time_summary = pd.DataFrame({
    "Time System": ["UTC", "UT1", "TAI", "GPS", "TT", "TDB"],
    "Time (hh:mm:ss)": [
        str(timedelta(seconds=utc)),
        str(timedelta(seconds=ut1)),
        str(timedelta(seconds=tai)),
        str(timedelta(seconds=gps)),
        str(timedelta(seconds=tt)),
        str(timedelta(seconds=tdb)),
    ]
})

display(HTML(time_summary.to_html(index=False)))
print("\nJulian centuries from J2000.0:")
print(f'{"TUT1:":<6} {tut1:.12f}')
print(f'{"TTT:":<6} {ttt:.12f}')
print(f'{"TTDB:":<6} {ttdb:.12f}')

Time System,Time (hh:mm:ss)
UTC,16:43:00
UT1,16:42:59.536674
TAI,16:43:32
GPS,16:43:13
TT,16:44:04.184000
TDB,16:44:04.185238



Julian centuries from J2000.0:
TUT1:  0.043674100545
TTT:   0.043674121031
TTDB:  0.043674121031
